In [1]:
import pdfplumber
import PyPDF2
import re
import spacy
from PyPDF2 import PdfReader
import pandas as pd
import nltk
import pickle

from nltk.tokenize import sent_tokenize

C:\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
C:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'
C:\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np

In [2]:

from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()

# df['lem_word']=[lemmatizer.lemmatize(word.lower()) for word in df['key']]
# df['lem_word'] = df['lem_word'].apply(lambda x: x.strip())

In [3]:
train_df = pd.read_excel('TRAINING_DATA.xlsx')
t=train_df.apply(tuple, axis=1)
out = list(map(eval, train_df['entity']))

In [4]:
training_data=[]
for i in range(len(train_df)):
    training_data.append((train_df['Train_data'][i],out[i]))

In [5]:
training_data

[('analysis and summary 6.1 .', {'entities': [(13, 20, 'R_T')]}),
 ('proportion of subject who develop fviii inhibitor \uf0b7 subject withfviii inhibitor confirmed by the central laboratory a a titer ≥0.6bu/mlat baseline ( ie , no longer at risk for inhibitor ) will not be included.\uf0b7 the number of subject who develop inhibitor ( numerator ) will include all subject where the blood sample collection for positive inhibitor occurred after the first dose of study drug , regardless of how many on-study ed had accrued at the time of sample collection .',
  {'entities': [(310, 316, 'R_T')]}),
 ('\uf0b7 the analysis will consist of the following descriptive statistic : numberof non- missingobservations , frequency and observed proportion of subject for which fviii inhibitor were detected at visit 4 .',
  {'entities': [(105, 114, 'R_T'), (57, 66, 'R_T')]}),
 ('a two-sided 90 % ci for the corresponding proportion will be computed using the exact method ( section5.2.1 ) .',
  {'entities': [(

In [6]:
from sklearn.model_selection import train_test_split
train,valid = train_test_split(training_data ,
                                   random_state=104, 
                                   test_size=0.30, 
                                   shuffle=True)

In [7]:
len(train)

359

In [8]:
len(valid)

155

In [9]:
# lists = [('They will be summarized using the on-study approach as defined in Section 3.5.1. The ontreatment+ 28-day approach will only be presented as a flag in listings.',{'entities':[(150, 158, 'R_L')]}),('Pre-dose (trough) PK concentrations will be listed using PK concentration set.',{'entities': [(44, 50, 'R_L')]}),('Laboratory test values for individual subjects will be listed.',{'entities': [(55, 61, 'R_L')]}),('All laboratory data listing and summary will be based on safety analysis set using the on-treatment+5-day approach.',{'entities': [(20, 27, 'R_L'),(32, 39, 'R_T')]}),('Deviations from the nominal time will be given in a separate listing.',{'entities': [(61, 68, 'R_L')]}),('a listing of all urine concentrations over each time interval sorted by group, subject ID and nominal collection duration postdose.',{'enities': [(2, 9, 'R_L')]}),('All prior and concomitant medication(s) as well as non-drug treatment(s) will be provided in listings.',{'entities': [(93, 100, 'R_L')]}),('Participant data listings for demography and baseline characteristics data will also be generated.',{'entities': [(17, 25, 'R_L')]}),('Participant data listings for demography and other infant data will also be generated.',{'entities': [(17, 25, 'R_L')]}),('The protocol deviations listings will be generated separately for maternal and infant participants.',{'entities': [(24, 32, 'R_L')]}),('Listings of participants reporting any AE and immediate AEs, and listings of all reported AEs will be generated.',{'entities': [(0, 8, 'R_L')]}),('Also, separate listings for participants with abnormal laboratory results will be generated.',{'entities': [(15, 23, 'R_L')]}),('Descriptive summaries (counts and percentages) and listings based on the safety population will be provided in accordance with the Pfizer reporting standards.',{'entities': [(51, 59, 'R_L'), (12, 21, 'R_T')]}),('The summaries and listings for these data will be generated only for maternal participants.',{'entities': [(4, 13, 'R_T'), (18, 26, 'R_L')]})]


In [10]:
# valid=valid+lists

In [11]:
def count_labels(dataset, labels):
    label_counts = {label: 0 for label in labels}
    for _, annotations in dataset:
        for _, _, label in annotations['entities']:
            if label in label_counts:
                label_counts[label] += 1
    return label_counts

 

labels_to_count = ['R_T', 'R_F', 'R_L']
label_counts = count_labels(train, labels_to_count)

 

print(label_counts)

{'R_T': 671, 'R_F': 41, 'R_L': 52}


In [12]:
#pip install imbalanced-learn

In [13]:
from collections import defaultdict
import random

def balance_data(training_data):
    # Count the frequency of each label
    label_counts = defaultdict(list)
    for sentence, annotations in training_data:
        for _, _, label in annotations['entities']:
            label_counts[label].append((sentence, annotations['entities']))

    # Find the label with the maximum frequency
    max_count = max(len(annotations_list) for annotations_list in label_counts.values())
    min_count = min(len(annotations_list) for annotations_list in label_counts.values())

    # Determine the target count for oversampling
    target_count = min_count + (max_count - min_count) // 2

    # Create a balanced training set
    balanced_data = []
    for label, annotations_list in label_counts.items():
        if len(annotations_list) == max_count:
            # Undersample the majority class to the target count
            undersampled_annotations = random.sample(annotations_list, target_count)
            balanced_data.extend(undersampled_annotations)
        else:
            # Oversample the minority classes to the target count
            oversampled_annotations = random.choices(annotations_list, k=target_count - len(annotations_list))
            balanced_data.extend(annotations_list + oversampled_annotations)

    balanced_training_data = [(sentence, {'entities': annotations}) for sentence, annotations in balanced_data]
    return balanced_training_data

balanced_training_data = balance_data(training_data)
#print(balanced_training_data)

  

In [14]:
def count_labels(dataset, labels):
    label_counts = {label: 0 for label in labels}
    for _, annotations in dataset:
        for _, _, label in annotations['entities']:
            if label in label_counts:
                label_counts[label] += 1
    return label_counts

 

labels_to_count = ['R_T', 'R_F', 'R_L']
label_counts = count_labels(balanced_training_data, labels_to_count)

 

print(label_counts)

{'R_T': 2904, 'R_F': 770, 'R_L': 597}


In [15]:
from __future__ import unicode_literals, print_function

import random
from pathlib import Path
import spacy
from tqdm import tqdm
from spacy.training.example import Example
import pickle

In [16]:


model = None
output_dir=Path("ner/")
n_iter=10


In [17]:

#load the model

if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")



Created blank 'en' model


In [18]:

#set up the pipeline

if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe('ner')
    #nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')
# adding labels to ner
for _, annotations in balanced_training_data:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])


In [19]:
example = []
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(balanced_training_data)
        losses = {}
        for text, annotations in tqdm(balanced_training_data):
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            nlp.update(
                [example], 
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)


  1%|          | 11/1503 [00:00<01:18, 18.93it/s]C:\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:152: UserWarning: [W030] Some entities could not be aligned in the text "kaplan-meier estimate ( product-limit estimate ) w..." with entities "[(204, 211, 'R_T'), (175, 181, 'R_T'), (242, 252, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
  2%|▏         | 30/1503 [00:02<02:35,  9.46it/s]C:\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:152: UserWarning: [W030] Some entities could not be aligned in the text "kaplan-meier estimate ( product-limit estimate ) w..." with entities "[(176, 183, 'R_T'), (122, 131, 'R_T'), (115, 117, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entit

 15%|█▌        | 226/1503 [00:11<00:52, 24.43it/s]C:\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:152: UserWarning: [W030] Some entities could not be aligned in the text "kaplan-meier estimate ( product-limit estimate ) w..." with entities "[(204, 211, 'R_T'), (175, 181, 'R_T'), (151, 160, ...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
 15%|█▌        | 229/1503 [00:11<00:58, 21.83it/s]C:\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:152: UserWarning: [W030] Some entities could not be aligned in the text "gmt of the rsv a–and rsv b–neutralizing antibody t..." with entities "[(172, 179, 'R_T'), (129, 131, 'R_T'), (92, 116, '...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  ent

 70%|██████▉   | 1045/1503 [01:23<00:46,  9.76it/s]C:\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:152: UserWarning: [W030] Some entities could not be aligned in the text "for each cohort , the endpoint defined in will be ..." with entities "[(23, 24, 'R_T'), (101, 108, 'R_T'), (70, 80, 'R_T...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
 72%|███████▏  | 1084/1503 [01:27<00:47,  8.80it/s]C:\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:152: UserWarning: [W030] Some entities could not be aligned in the text "a summary of duration ( day ) of dosinginterruptio..." with entities "[(2, 9, 'R_T'), (20, 21, 'R_T')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] 

{'ner': 3812.2998677915884}


100%|██████████| 1503/1503 [02:12<00:00, 11.32it/s]


...

{'ner': 708.1909437096365}


100%|██████████| 1503/1503 [01:22<00:00, 18.13it/s]


{'ner': 713.9284035145153}


100%|██████████| 1503/1503 [01:23<00:00, 18.06it/s]

{'ner': 637.1991262765455}


In [20]:
lists=[('They will be summarized using the on-study approach as defined in Section 3.5.1. The ontreatment+ 28-day approach will only be presented as a flag in listings.',{'entities':[(150, 158, 'R_L')]}),
       ('Pre-dose (trough) PK concentrations will be listed using PK concentration set.',{'entities': [(44, 50, 'R_L')]}),
       ('Laboratory test values for individual subjects will be listed.',{'entities': [(55, 61, 'R_L')]}),
       ('All laboratory data listing and summary will be based on safety analysis set using the on-treatment+5-day approach.',{'entities': [(20, 27, 'R_L'),(32, 39, 'R_T')]}),
       ('Deviations from the nominal time will be given in a separate listing.',{'entities': [(61, 68, 'R_L')]}),
       ('a listing of all urine concentrations over each time interval sorted by group, subject ID and nominal collection duration postdose.',{'entities': [(2, 9, 'R_L')]}),
        ('All prior and concomitant medication(s) as well as non-drug treatment(s) will be provided in listings.',{'entities': [(93, 100, 'R_L')]}),
        ('Participant data listings for demography and baseline characteristics data will also be generated.',{'entities': [(17, 25, 'R_L')]}),
       ('Participant data listings for demography and other infant data will also be generated.',{'entities': [(17, 25, 'R_L')]}),
        ('The protocol deviations listings will be generated separately for maternal and infant participants.',{'entities': [(24, 32, 'R_L')]}),
       ('Listings of participants reporting any AE and immediate AEs, and listings of all reported AEs will be generated.',{'entities': [(0, 8, 'R_L')]}),
         ('Also, separate listings for participants with abnormal laboratory results will be generated.',{'entities': [(15, 23, 'R_L')]}),
         ('Descriptive summaries (counts and percentages) and listings based on the safety population will be provided in accordance with the Pfizer reporting standards.',{'entities': [(51, 59, 'R_L'), (12, 21, 'R_T')]}),
         ('The summaries and listings for these data will be generated only for maternal participants.',{'entities': [(4, 13, 'R_T'), (18, 26, 'R_L')]})]
       

In [21]:
valid_ls=valid+lists

In [22]:
examples = []

for text, annots in valid_ls:
    doc = nlp.make_doc(text)
    examples.append(Example.from_dict(doc, annots))
print(nlp.evaluate(examples))

C:\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:152: UserWarning: [W030] Some entities could not be aligned in the text "gmt before and 1 month after vaccination 1 and ass..." with entities "[(66, 73, 'R_T'), (32, 34, 'R_T'), (82, 92, 'R_T')...". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
C:\Anaconda3\lib\site-packages\spacy\training\iob_utils.py:152: UserWarning: [W030] Some entities could not be aligned in the text " the tipping point analysis ( section 5.3.1 ) wil..." with entities "[(224, 229, 'R_T'), (550, 562, 'R_T')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  entities=ent_str[:50] + "..." if len(ent_str) > 50 else ent_str,
C:\Anaconda3\lib\site-packages\spacy\training\iob_uti

{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0, 'ents_p': 0.9326923076923077, 'ents_r': 0.9208860759493671, 'ents_f': 0.9267515923566879, 'ents_per_type': {'R_T': {'p': 0.9315589353612167, 'r': 0.9386973180076629, 'f': 0.9351145038167938}, 'R_F': {'p': 0.8235294117647058, 'r': 0.9333333333333333, 'f': 0.8749999999999999}, 'R_L': {'p': 1.0, 'r': 0.8, 'f': 0.888888888888889}}, 'speed': 21547.82776874868}


In [23]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)
pickle.dump(nlp, open( "ner_nlp_Model_2.pkl", "wb" ))


Saved model to ner


In [24]:
text="Descriptive statistics (number and frequency) will be provided for all adverse event summaries."
word_list = nltk.word_tokenize(text)
lem_line= ' '.join([lemmatizer.lemmatize(w.lower()) for w in word_list])

In [25]:
doc=nlp(lem_line)
print(doc)
for ent in doc.ents:
    print(ent.label_+ '  ------>   ' + ent.text)

descriptive statistic ( number and frequency ) will be provided for all adverse event summary .
R_T  ------>   statistic
R_T  ------>   number and frequency
R_T  ------>   summary


In [26]:
#text="Bar charts with the proportions of subjects reporting each systemic event within 14 days after vaccination will be plotted for each vaccine group. The bars will be stacked by maximum severity to highlight the proportions of subjects by severity."
text="The number and percentage of subject with at least 1 diagnosis , overall and at each SOC and PTlevel , maybe summarized by vaccine group and for overall subject included in the safety population for each cohort"
word_list = nltk.word_tokenize(text)
lem_line= ' '.join([lemmatizer.lemmatize(w.lower()) for w in word_list])

In [27]:
doc=nlp(lem_line)
print(doc)
for ent in doc.ents:
    print(ent.label_+ '  ------>   ' + ent.text)

the number and percentage of subject with at least 1 diagnosis , overall and at each soc and ptlevel , maybe summarized by vaccine group and for overall subject included in the safety population for each cohort
R_T  ------>   number and percentage
R_T  ------>   summarized


In [28]:
text= ['Descriptive statistics (number and frequency) will be provided for all adverse event summaries.'
        'following demographic and baseline characteristics will be summary by number',
        'The number and proportion of participants experiencing DLTs during the DLT-evaluation period specific to the DLTs will be summarized by SOC and PT based on maximum toxicity grade by dose cohort. Analyses of DLT will be performed on DLT Evaluable Set. ',
        'A listing of elranatamab, iberdomide concentrations by study visit and timepoint will be generated.',
        'Bar charts with the proportions of subjects reporting each systemic event within 14 days after vaccination will be plotted for each vaccine group. The bars will be stacked by maximum severity to highlight the proportions of subjects by severity.',
        'The percentage of participants with positive ADA will be summarized.',
       ' Listings and summary tabulations of the ADA data at baseline and post-baseline will be generated.',
      'Figures : Bar chart with the proportion of subject reporting each AE category after vaccination will be plotted for each vaccine']

In [29]:
for each in text:
    sentence = sent_tokenize(each)
    for sen in sentence:
        print(sen,'\n')
        word_list = nltk.word_tokenize(sen)
        lem_line= ' '.join([lemmatizer.lemmatize(w) for w in word_list])
        doc=nlp(lem_line)
        print(sen)
        for ent in doc.ents:
            print(ent.label_+ '  ------>   ' + ent.text)

Descriptive statistics (number and frequency) will be provided for all adverse event summaries.following demographic and baseline characteristics will be summary by number 

Descriptive statistics (number and frequency) will be provided for all adverse event summaries.following demographic and baseline characteristics will be summary by number
R_T  ------>   statistic
R_T  ------>   number and frequency
R_T  ------>   summary
The number and proportion of participants experiencing DLTs during the DLT-evaluation period specific to the DLTs will be summarized by SOC and PT based on maximum toxicity grade by dose cohort. 

The number and proportion of participants experiencing DLTs during the DLT-evaluation period specific to the DLTs will be summarized by SOC and PT based on maximum toxicity grade by dose cohort.
R_T  ------>   number and proportion
R_T  ------>   summarized
R_T  ------>   maximum
Analyses of DLT will be performed on DLT Evaluable Set. 

Analyses of DLT will be performed 

In [30]:
for each in text:
    sentence = sent_tokenize(each)
    for sen in sentence:
        word_list = nltk.word_tokenize(sen)
        lem_line= ' '.join([lemmatizer.lemmatize(w.lower()) for w in word_list])
        doc=nlp(lem_line)
        color={'R_T':'pink', 'R_L':'cyan', 'R_F':'violet'}
        options={'ents':['R_T','R_L','R_F'],'colors':color}
        spacy.displacy.render(doc,style='ent',jupyter=True,options=options)
        

C:\Anaconda3\lib\site-packages\spacy\displacy\__init__.py:213: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)
